In [1]:
import glob
import os
import pandas as pd
from rdkit import Chem
from Mol2Writer import MolToMol2File

In [24]:
# 1st convert all .sdf to .mol2 format
for path in glob.glob('../data/optimized_ligands/*/*'):
    mol=Chem.SDMolSupplier(path)[0]
    MolToMol2File(mol, filename=path[:-3]+'mol2')

for path in glob.glob('../data/reference_ligands/*/*'):
    mol=Chem.SDMolSupplier(path)[0]
    MolToMol2File(mol, filename=path[:-3]+'mol2')

In [2]:
def DockRMSD(query, template):
    '''returns the RMSD between the docked pose and the confirmed pose.'''
    os.system('/home/fol007/DockRMSD/DockRMSD '+query+' '+template+' > output.txt')
    try:
        file = open('output.txt', 'r')
        for line in file:
            if 'Calculated Docking RMSD:' in line:
                line = line.split('\n')[0]
                line = line.split(': ')[1]
                return float(line)
    except:
        return None

In [16]:
dictionary = {'complex':[], 'rmsd':[]}

for path_opt in glob.glob('../data/optimized_ligands/*/*.mol2'):

    complex_id = path_opt.split('/')[-1].split('.')[0]

    path_ori = path_opt.split('/')
    path_ori[2] = 'reference_ligands'
    path_ori = '/'.join(path_ori)

    dictionary['complex'] += [complex_id]
    rmsd = DockRMSD(path_opt, path_ori)
    dictionary['rmsd'] += [rmsd]


In [17]:
df = pd.DataFrame(dictionary)

In [30]:
df[df['rmsd'].isna()]

,complex,rmsd
1344,4OAR_2S0,NaN


In [29]:
p1 = '../data/optimized_ligands/P22906/1M7A_MQU.mol2'
p2 = '../data/reference_ligands/P22906/1M7A_MQU.mol2'

rmsd=DockRMSD(p1, p2)
print(rmsd)
df.loc[df['complex']=='1M7A_MQU', 'rmsd'] = rmsd

0.335


In [31]:
p1 = '../data/optimized_ligands/P06401/4OAR_2S0.mol2'
p2 = '../data/reference_ligands/P06401/4OAR_2S0.mol2'

rmsd=DockRMSD(p1, p2)
print(rmsd)
df.loc[df['complex']=='4OAR_2S0', 'rmsd'] = rmsd

0.233


In [40]:
print(df.shape[0])
print('rmsd bellow 1 , rmsd above 1')
print(df[df['rmsd']<1].shape[0], df[df['rmsd']>1].shape[0])

1991
rmsd bellow 1 , rmsd above 1
1754 237


In [41]:
expanded_astex_df = pd.read_csv('../data/expanded_Astex_with_smiles.csv')
expanded_astex_df['hyde_rmsd'] = None

In [43]:
for complex, rmsd in df[['complex', 'rmsd']].values:
    Protein_ID = complex.split('_')[0]
    Ligand_Name = complex.split('_')[1]
    expanded_astex_df.loc[(expanded_astex_df['Protein_ID']==Protein_ID) &
                          (expanded_astex_df['Ligand_Name']==Ligand_Name), 'hyde_rmsd'] = rmsd

In [47]:
expanded_astex_df.to_csv('../data/expanded_Astex_with_smiles.csv', index=False)